In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sms-spam-collection-dataset/spam.csv


#### Import libraries

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import re
from nltk.corpus import stopwords
import string
from keras.utils import to_categorical
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
tqdm.pandas()

Using TensorFlow backend.


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Read Data

In [3]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', engine='python',usecols=range(2),names=['class','text'], skiprows=[0])
df.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
class    5572 non-null object
text     5572 non-null object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df['class'] = df['class'].rank(method='dense', ascending=False).astype(int)

### Text Processing

In [6]:
def clean_data(text):
   lower = text.lower()
   splitted = lower.split()
   re_punc = re.compile('[%s]' % re.escape(string.punctuation))
   tokens = [re_punc.sub('',w) for w in splitted]
   tokens = [word for word in tokens if word.isalpha()]
   stop_words = set(stopwords.words('english'))
   tokens = [w for w in tokens if not w in stop_words]
   lemmeted = [WordNetLemmatizer().lemmatize(w) for w in tokens]
   tokens = [word for word in lemmeted if len(word) > 2]
   return tokens

x_train, x_val, y_train, y_val = train_test_split(df['text'], df['class'])
vocab = Counter()
for index, row in x_train.iteritems():
  vocab.update(clean_data(row))

In [7]:
vocab.most_common(20)

[('call', 451),
 ('get', 300),
 ('free', 212),
 ('dont', 209),
 ('know', 202),
 ('ltgt', 190),
 ('ill', 188),
 ('like', 182),
 ('got', 180),
 ('day', 179),
 ('good', 176),
 ('come', 175),
 ('time', 172),
 ('text', 159),
 ('love', 159),
 ('want', 158),
 ('one', 147),
 ('lor', 133),
 ('need', 133),
 ('going', 127)]

In [8]:
vocab_size = len(vocab)
print("Vocabulary Size is: ", vocab_size)

Vocabulary Size is:  6274


### term frequency–inverse document frequency (tf-idf)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(vocabulary=vocab.keys())
x_train = v.fit_transform(x_train)
x_val = v.fit_transform(x_val)

### Model Preparation

In [10]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

model = BernoulliNB()
model.fit(x_train, y_train)
pred = model.predict(x_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           1       0.99      0.79      0.88       198
           2       0.97      1.00      0.98      1195

    accuracy                           0.97      1393
   macro avg       0.98      0.89      0.93      1393
weighted avg       0.97      0.97      0.97      1393

